<a href="https://colab.research.google.com/github/amkorousagi/PLA_AI/blob/main/PLA_tenacity_elongation_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import r2_score

In [ ]:

# 목적 변수 둘 중 하나는 제거하고 호출해야함
def filter_data(origin, criteria, target):
  ccnt = 0
  result = pd.DataFrame([],columns=origin.columns)
  sum = 0
  cnt = 0
  lastidx = 0
  prior = origin.iloc[0]
  l = list()
  for idx, row in origin.iterrows():
    same = True
    lastidx = idx
    for hname in origin.columns.values:
      if hname != target:
        if row[hname] != prior[hname]:
          same = False
          break
    if same == True:
      sum += row[target]
      cnt += 1
      l.append(row)
    else:
      mean = sum/cnt
      for i in range(cnt):
        if abs(origin[target].iloc[idx-i-1]-mean) >= criteria:
          #print("mean",mean)
          #print("cnt",cnt)
          #print("mean ", mean, ",data ", abs(origin[target].iloc[idx-i-1]-mean))
          #print(l)
          #print('\n')
          ccnt += 1
        else:
          result = result.append(origin.iloc[idx-i-1], ignore_index=True)
      sum = row[target]
      cnt = 1
      prior = row
      l=list()
      l.append(row)
  
  mean = sum/cnt
  for i in range(cnt):
    if abs(origin[target].iloc[lastidx-i]-mean) >= criteria:
      ccnt += 1
    else:
      result = result.append(origin.iloc[idx-i], ignore_index=True)
  print("ro :",ccnt)
  return result

In [ ]:
# rs 0-9 : [0.15808540493249895, 0.14105506174266338, 0.1516697186231613, 0.1559910671412945, 0.16005230233073234, 0.16345584884285927, 0.15981756284832954, 0.1577516482770443]
# rs 10-19 : 
target = "Tenacity"
ntarget = "Elongation"

data = pd.read_excel('process_data.xlsx')
data = data.drop([ntarget], axis=1)
data = data.drop('no', axis=1)
origin_data = data
mmm = []
mmae_list = []
cccnt = 0
cr = []
rs = []
for i in range(8):
  for j in range(10):
    rs.append(j+10)
    if i==0:
      cr.append(100)
    else:
      cr.append(i*0.1+0.2)

print(cr)
print(rs)
for i in range(80):
  data = origin_data
  data = filter_data(data, cr[i], target)

  #print(data)

  train_dataset = data.sample(frac=0.9, random_state=rs[i])
  test_dataset = data.drop(train_dataset.index)

  train_stats = train_dataset.describe()
  train_stats.pop(target)
  train_stats = train_stats.transpose()

  train_labels = train_dataset.pop(target)
  test_labels = test_dataset.pop(target)

  def norm_minmax(x):
    return (x - train_stats['min']) / (train_stats['max'] - train_stats['min'])

  def norm_distribution(x):
    return (x - train_stats['mean']) / train_stats['std']
  
  train_data = norm_minmax(train_dataset)
  test_data = norm_minmax(test_dataset)

  def build_l2_model():
    model = keras.Sequential([
                              layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=[len(train_data.keys())]),
                              # layers.Dense(64, kernel_regularizer=keras.regularizers.l1(0.001), activation='relu', input_shape=[len(train_data.keys())]),
                              # layers.Dropout(0.5),
                              layers.Dense(64, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'),
                              # layers.Dense(64, kernel_regularizer=keras.regularizers.l1(0.001), activation='relu'),
                              # layers.Dropout(0.5),
                              layers.Dense(1)
    ])

    # optimizer = tf.keras.optimizers.RMSprop(0.001)
    optimizer = tf.keras.optimizers.Adam(0.001)

    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae', 'mse'])
    return model

  EPOCHS = 1000

  model = build_l2_model()

  mae_list = list()
  mse_list = list()

  for j in range(10):
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    history = model.fit(train_data, train_labels, epochs=EPOCHS, validation_split=0.2, verbose=0, callbacks=[early_stop, PrintDot()])
    loss, mae, mse = model.evaluate(test_data, test_labels, verbose=2)
    mae_list.append(mae)
    mse_list.append(mse)
  cccnt +=1
  print(cr[i],", ",rs[i]," : ",sum(mae_list)/len(mae_list))
  mmae_list.append(sum(mae_list)/len(mae_list))
  if cccnt % 10 == 0:
    print(cr[i]," : ",sum(mmae_list)/len(mmae_list))
    mmm.append(sum(mmae_list)/len(mmae_list))
    mmae_list = []



[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.30000000000000004, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.9000000000000001, 0.9000000000000001, 0.9000000000000001, 0.9000000000000001, 0.9000000000000001, 0.9000000000000001, 0.9000000000000001, 0.9000000000000001, 0.9000000000000001, 0.9000000000000001]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 11, 12, 13, 14

In [ ]:
print(mmm)

[0.16653846397995947, 0.1398153717070818, 0.14584364227950575, 0.14777433194220063, 0.150052424967289, 0.16457318276166916, 0.166081540286541, 0.1676209408044815]
